<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [1]:
!pip install pdfminer.six
!pip install spacy
!pip install tabula-py
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 5.6MB 4.9MB/s 
     |████████████████████████████████| 13.7MB 285kB/s 


## Upload PDF and Json

In [6]:
from google.colab import files

uploaded = files.upload()


Saving 0070C022_220C0700.json to 0070C022_220C0700.json
Saving 0340C022_220C0430.json to 0340C022_220C0430.json
Saving 0510C022_220C0150.json to 0510C022_220C0150.json
Saving 0540C022_220C0450.json to 0540C022_220C0450.json
Saving 0620C022_220C0260.json to 0620C022_220C0260.json
Saving 0810C022_220C0180.json to 0810C022_220C0180.json
Saving 0820C022_220C0280.json to 0820C022_220C0280.json
Saving 0900C022_220C0090.json to 0900C022_220C0090.json
Saving 0940C022_220C0490.json to 0940C022_220C0490.json
Saving 0960C022_220C0690.json to 0960C022_220C0690.json
Saving 0980C022_220C0890.json to 0980C022_220C0890.json
Saving 1040C022_220C0401.json to 1040C022_220C0401.json
Saving 1080C022_220C0801.json to 1080C022_220C0801.json
Saving 1090C022_220C0901.json to 1090C022_220C0901.json
Saving 1200C022_220C0021.json to 1200C022_220C0021.json
Saving 1270C022_220C0721.json to 1270C022_220C0721.json
Saving 1360C022_220C0631.json to 1360C022_220C0631.json
Saving 1380C022_220C0831.json to 1380C022_220C08

In [0]:
ls

dataset.json  sample_data/


## Table PDF

In [0]:
from tabula import read_pdf
import pandas as pd

def extract_tables(filename):
  dfs = read_pdf(list_pdf[0], pages='all')
  tables_str = []
  for df in dfs:
  if not df.empty:
    table_str = ""
    for c in df.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in df.iterrows():
      for c in df.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    tables_str.append(table_str)
  return dfs, tables_str

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



## Extract PDF text

In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

## Load PDF and Json

In [33]:
import json
import os 

data = {}
data_label = {}
data_no_label = {}

for f in uploaded.keys():
  try:
    name, ext = os.path.splitext(f)
    if ext == '.pdf' and name not in pdfs:
      text=extract_text_from_pdf(f)
      if "Déclaration de franchissement de seuils" in text:
        x =  {"file": f, "text": text}
        data[name] = x
        data_no_label[name] = x
    elif ext == '.json':
      with open(f) as fd:
        x = json.load(fd)
        data[name] = x
        if 'entities' in x and len(x['entities']) != 0:
          data_label[name] = x
          if name in data_no_label:
            del data_no_label[name]
        else:
          data_no_label[name] = x
  except Exception as e:
    print("Error : ", f)
    print(e)
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

117 texts with 117 labeled and 0 to label


## Labellisation

In [0]:
import IPython
from google.colab import output
import os

class AnnotWidget():
  def __init__(self, to_annot, start=0, model=None):
    self.to_annot = to_annot
    self.cur = start
    self.model = model
    self.colors_labels = [
        ("rgb(255, 0, 0)", "ISSUER", 'A'),
        ("rgb(255, 0, 255)", "FIRM", 'Z'),
        ("rgb(0, 255, 255)", "NUMBER", 'E'),
        ("rgb(160, 160, 255)", "PERCENT", 'R'),
        ("rgb(192, 192, 192)", "TYPE", 'T'),
        ("rgb(255, 255, 0)", "DIRECTION", 'S'),
        ("rgb(0, 255, 0)", "DATE", 'D')
      ]
    self.controls = ""
    for c, l, k in self.colors_labels:
      self.controls += '''<button id="key_'''+k+'''" style="background-color: '''+\
        c+''';" onclick='document.execCommand("backColor", false, "'''+c+'''")'>'''+l+" ("+k+")</button>"

  def text_entities_to_html(self):
    if 'entities' not in self.to_annot[self.cur] or len(self.to_annot[self.cur]['entities']) == 0:
      return self.to_annot[self.cur]["text"]
    res = ""
    offset = 0
    text = self.to_annot[self.cur]["text"]
    for ent_s, ent_e, label in self.to_annot[self.cur]['entities']:
      res += text[offset:ent_s]
      res += '''<span style="background-color: '''+ self.get_color(label) + '''">'''
      res += text[ent_s:ent_e]
      res += "</span>"
      offset = ent_e
    res += text[offset:]
    return res

  def run(self):
    display(IPython.display.HTML('''
    <p>'''+str(self.cur) + "/" + str(len(self.to_annot)) + '''</p>
    <div id="ed" contenteditable="true">'''+self.text_entities_to_html()+'''</div>
    <fieldset>
    '''+ self.controls +'''
      <button id=""key_Q" onclick="document.execCommand('removeFormat', false, null)">remove (Q)</button>
      <button onclick="google.colab.kernel.invokeFunction('notebook.Prediction');">pred</button>
      <button id='prev'>prev</button>
      <button id='next'>next</button>
    </fieldset>
    <script>
      function change(dir) {
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, rgb])
          }
          offset = next_offset;
        }
        google.colab.kernel.invokeFunction('notebook.SaveEntities', [res, dir]);
        document.querySelector('#ed').innerHTML = next_text;
      };
      document.querySelector('#prev').onclick = () => {
        change(-1);
      };
      document.querySelector('#next').onclick = () => {
        change(1);
      };
      document.querySelector('#ed').onkeydown = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeypress = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeyup = (ev) => {
        ev.preventDefault();
        button_id = "#key_" + String.fromCharCode(ev.keyCode);
        button = document.querySelector(button_id)
        if (button) {
          button.click();
        }
      };
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)
    output.register_callback('notebook.Console', self.console)
    output.register_callback('notebook.Prediction', self.prediction)

  def get_color(self, label):
    for c, l, _ in self.colors_labels:
        if l == label:
          return c

  def get_label(self, color):
    for c, l, _ in self.colors_labels:
        if c == color:
          return l

  def save_entities(self, res, inc=1):
    self.to_annot[self.cur]['entities'] = []
    for ent_s, ent_e, color in res:
      self.to_annot[self.cur]['entities'].append((ent_s, ent_e, self.get_label(color)))
    if len(self.to_annot[self.cur]['entities']) != 0:
      name, _ = os.path.splitext(self.to_annot[self.cur]['file'])
      with open(name+'.json', 'w+') as f:
        json.dump(self.to_annot[self.cur], f)
    print("hello")
    output.clear()
    if (self.cur < len(self.to_annot) and inc == 1) or (self.cur > 0 and inc == -1):
      self.cur += inc
    self.run()

  def console(self, s):
    print(s)

  def prediction(self):
    if (self.model is not None) and (('entities' not in self.to_annot[self.cur]) or (len(self.to_annot[self.cur]['entities']) == 0)):
      doc = self.model(self.to_annot[self.cur]["text"])
      self.to_annot[self.cur]['entities'] = []
      for ent in doc.ents:
        self.to_annot[self.cur]['entities'].append([ent.start_char, ent.end_char, ent.label_])
      output.clear()
      self.run()


In [0]:
w = AnnotWidget(data_no_label, start=0, model=None)

In [14]:
w.run()

In [0]:
data = {x['file'] : x for x in data}
data_label = {x['file'] : x for x in data_label}
data_no_label = {x['file'] : x for x in data_no_label}
for x in w.to_annot:
  if 'entities' in x and len(x['entities']) != 0:
    data[x['file']] = x
    data_label[x['file']] = x
    del data_no_label[x['file']]
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

## Download Json

In [0]:
!zip dataset.zip *.json

updating: 0070C022_220C0700.json (deflated 66%)
updating: 0340C022_220C0430.json (deflated 60%)
updating: 0510C022_220C0150.json (deflated 51%)
updating: 0540C022_220C0450.json (deflated 62%)
updating: 0620C022_220C0260.json (deflated 50%)
updating: 0810C022_220C0180.json (deflated 69%)
updating: 0820C022_220C0280.json (deflated 60%)
updating: 0900C022_220C0090.json (deflated 74%)
updating: 0940C022_220C0490.json (deflated 57%)
updating: 0960C022_220C0690.json (deflated 69%)
updating: 0980C022_220C0890.json (deflated 54%)
updating: 1040C022_220C0401.json (deflated 61%)
updating: 1080C022_220C0801.json (deflated 51%)
updating: 1090C022_220C0901.json (deflated 67%)
updating: 1200C022_220C0021.json (deflated 57%)
updating: 1270C022_220C0721.json (deflated 64%)
updating: 1360C022_220C0631.json (deflated 65%)
updating: 1380C022_220C0831.json (deflated 55%)
updating: 1430C022_220C0341.json (deflated 67%)
updating: 1500C022_220C0051.json (deflated 59%)
updating: 1550C022_220C0551.json (deflat

In [0]:
from google.colab import files
files.download('dataset.zip') 

## Train NER

In [0]:
train_data = []
for x in data_label:
  entities = [tuple(ent) for ent in  x["entities"]]
  train_data.append((x['text'], {'entities' : entities}))
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE']
test_data = train_data[:20]
train_data = train_data[20:]

In [36]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=40
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  # batch up the examples using spaCy's minibatch
  for itn in range(n_iter):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=sizes)
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
    print("Losses", losses)
  score = nlp.evaluate(test_data)

Losses {'ner': 23964.736129675963}
Losses {'ner': 21973.9982841108}
Losses {'ner': 21204.750617760612}
Losses {'ner': 20787.01534249168}
Losses {'ner': 20451.064186168485}
Losses {'ner': 20219.935827415058}
Losses {'ner': 20128.03605991768}
Losses {'ner': 19915.52559884684}
Losses {'ner': 20032.747798833996}
Losses {'ner': 19953.6941755265}
Losses {'ner': 19843.57615134865}
Losses {'ner': 19724.049177646637}
Losses {'ner': 19941.698392510414}
Losses {'ner': 19831.664188325405}
Losses {'ner': 19661.403124719858}
Losses {'ner': 19561.296861588955}
Losses {'ner': 19658.20818388462}
Losses {'ner': 19976.871670246124}
Losses {'ner': 19725.043011188507}
Losses {'ner': 20156.993488043547}
Losses {'ner': 19920.45144336205}
Losses {'ner': 20021.656602740288}
Losses {'ner': 20174.712134584785}
Losses {'ner': 20607.4815261364}
Losses {'ner': 19799.648339748383}
Losses {'ner': 20812.619098186493}
Losses {'ner': 19801.717925186153}
Losses {'ner': 20950.092694044113}
Losses {'ner': 20289.96589791774

In [37]:
print(score.scores)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 92.62295081967213, 'ents_r': 94.9579831932773, 'ents_f': 93.7759336099585, 'ents_per_type': {'ISSUER': {'p': 88.0, 'r': 100.0, 'f': 93.61702127659575}, 'DIRECTION': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'DATE': {'p': 90.47619047619048, 'r': 95.0, 'f': 92.68292682926828}, 'TYPE': {'p': 93.10344827586206, 'r': 100.0, 'f': 96.42857142857143}, 'FIRM': {'p': 100.0, 'r': 68.75, 'f': 81.48148148148148}, 'PERCENT': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'NUMBER': {'p': 86.66666666666667, 'r': 100.0, 'f': 92.85714285714286}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}


In [38]:
from spacy import displacy
doc = nlp(data[0]['text'])
displacy.render(doc, style='ent', jupyter=True)

In [39]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)

220C0700-FR0000034639-FS0185            220C0700 NP 220C0700-FR0000034639-FS0185
FR0000034639-FS0185 21 février 2020    NP FR0000034639-FS0185
Déclaration de franchissement de seuils (article L. 233 NP Déclaration
7 du code de commerce)   ALTRAN TECHNOLOGIES  (Euronext Paris)   Par courrier NP code
le 21 février NP février
la société Bank of America Corporation (Corporation Trust Center, Orange Street, Wilmington, DE 19801, Etats-Unis) a déclaré avoir franchi en hausse, indirectement par l’intermédiaire des sociétés qu’elle contrôle NP déclaré
les seuils de 5% du capital et des droits de vote de la société ALTRAN TECHNOLOGIES et détenir 14 377 895 actions ALTRAN TECHNOLOGIES NP seuils
de droits de vote NP droits
et 5,58% des droits de vote de cette société1, répartis NP %
Actions NP Actions
% NP %
de vote NP vote
droits de vote Merrill NP droits
Lynch NP Lynch
Europe NP Europe
0,11 BofA Securities NP BofA
213 ns 213 ns Total Bank of America NP America
Ce franchissement de seuils résult